In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re


In [2]:
ds = pd.read_csv('./log_datasets/hourly_apple_alt.csv')
ds = ds.dropna()
ds

,Unnamed: 0,text,dir,dtime
0,0,think.invest.appl.alibaba.netflix.nvidia.tesla...,1.0,2018-01-02 09:31:00
1,1,store.jam.california.start.recreat.pot.sale.as...,1.0,2018-01-02 00:00:00
2,2,smart.lock.compani.otto.suspend.oper.fail.acqu...,1.0,2018-01-02 07:49:00
3,3,isnt.microsoft.nasdaqmsft.year.ago.compani.mad...,1.0,2018-01-02 00:00:00
4,4,paul.r.la.monica.lamonicabuzz.januari.et.stock...,1.0,2018-01-02 11:30:00
...,...,...,...,...
59097,59097,follow.stock.news.edit.follow.benjamin.rain.ed...,-1.0,2019-02-01 19:02:00
59098,59098,post.hayley.millar.appl.inc.nasdaqaapl.–.equit...,-1.0,2019-02-01 18:56:00
59099,59099,amazon.q.earn.breakdown.investor.worri.slow.gr...,-1.0,2019-02-01 19:02:00
59100,59100,post.hayley.millar.appl.nasdaqaapl.‘.stock.“.n...,-1.0,2019-02-01 18:56:00


In [3]:
news_data = pd.read_csv('./datasets/news_data_filtered_unique.csv')
news_data = news_data.dropna()
news_data

,company,date,text,file,time
0,aapl,2018-08-13,kayla.tausch.nightli.busi.report.correspond.tw...,../news/2018_08_d157b48c57be246ec7dd80e7af4388...,00:00:00
1,aapl,2018-08-31,edgewood.manag.llc.lower.stake.appl.inc.nasdaq...,../news/2018_08_d157b48c57be246ec7dd80e7af4388...,09:13:00
2,aapl,2018-08-27,dow.jump.point.nasdaq.hit.presid.trump.announc...,../news/2018_08_d157b48c57be246ec7dd80e7af4388...,19:27:00
3,aapl,2018-08-01,south.korea.market.may.give.support.point.pm.e...,../news/2018_08_d157b48c57be246ec7dd80e7af4388...,23:01:00
4,aapl,2018-08-19,tim.cook.app.justin.sullivangetti.imag.might.s...,../news/2018_08_d157b48c57be246ec7dd80e7af4388...,20:00:00
...,...,...,...,...,...
64875,aapl,2018-12-27,googl.hardwar.ambit.start.pay.evan.niu.cfa.mot...,../news/2018_12_d157b48c57be246ec7dd80e7af4388...,00:23:00
64876,aapl,2018-12-13,etf.preview.etf.futur.point.higher.street.dige...,../news/2018_12_d157b48c57be246ec7dd80e7af4388...,14:46:00
64877,aapl,2018-12-06,oneday.break.stock.market.continu.drop.thursda...,../news/2018_12_d157b48c57be246ec7dd80e7af4388...,00:00:00
64878,aapl,2018-12-14,laurel.wealth.advisor.inc.lift.posit.appl.inc....,../news/2018_12_d157b48c57be246ec7dd80e7af4388...,14:44:00


In [4]:
news_data['text'] = news_data['text'].apply(lambda x: x.replace(".", " "))
ds['text'] = ds['text'].apply(lambda x: x.replace(".", " "))


In [5]:
r = re.compile("(apple)|(aapl)|(appl)")
mask_text = ds['text'].apply(lambda x: bool(r.match(x)))

In [6]:
ds = ds[mask_text]
ds.shape

(5694, 4)

In [7]:
vectorizer = TfidfVectorizer( max_features=16000)
vectorizer.fit(news_data['text'])
features = vectorizer.transform(ds['text'])

KeyboardInterrupt: 

In [7]:
vectorizer = CountVectorizer()
vectorizer.fit(news_data['text'])
features = vectorizer.transform(ds['text'])

In [46]:
vectorizer_bin = CountVectorizer(binary=True, max_features=16000)
vectorizer_bin.fit(news_data['text'])
features_bin = vectorizer_bin.transform(ds['text'])

In [8]:
labels = ds['dir'].to_numpy()

In [9]:
from sklearn.feature_selection import chi2

In [10]:
chi2_test = chi2(features,labels)
mask = chi2_test[1] <= 0.01

In [11]:
chi2_features = features[:,mask]
chi2_features.shape

(5694, 2159)

In [12]:
f_train, f_test, y_train, y_test = train_test_split(chi2_features, labels, test_size=0.2, random_state=42)

In [13]:
clf = LogisticRegression(penalty='l1', solver='liblinear').fit(f_train, y_train)

In [14]:
clf.score(f_test,y_test)
#0.6619841966637401

0.6619841966637401

In [19]:
just_up = np.ones(y_test.shape)

correct = sum(just_up == y_test)

print(correct/len(just_up))


0.5513608428446005


In [72]:
amazon_ds = pd.read_csv('./log_datasets/AMAZON_text_hourly.csv')

In [81]:
features = vectorizer.transform(amazon_ds['text'])
labels = amazon_ds['dir'].to_numpy()
f_train, f_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
clf = LogisticRegression(penalty='l1', solver='liblinear').fit(f_train, y_train)
clf.score(f_test,y_test)

0.5876726886291179

In [15]:
from sklearn.naive_bayes import GaussianNB

In [16]:
clf = GaussianNB()

In [17]:
clf.fit(f_train.toarray(),y_train)

GaussianNB()

In [18]:
clf.score(f_test.toarray(), y_test)

0.485513608428446